In [1]:
from collections import defaultdict

f = open("dataset_865533_10.txt","r")
Text = f.readline().strip('\n')
Patterns = [str(x) for x in next(f).split()]
d = int(f.readline())
f.close()

def MultipleApproximatePatternMatching(Text, Patterns, d):
    if Text[-1] != '$':
        Text = Text + "$"
    suffix_array = SuffixArray(Text)
    print(suffix_array)
    Transform = BWTfromSuffixArray(Text, suffix_array)
    
    PatternMatchIndices = defaultdict(set)
    
    match_candidates = BetterBWMatchingPatterns(Transform, Patterns, d)
    
    for i in range(len(Patterns)):
        if match_candidates[i] != 0:
            Pattern = Patterns[i]
            ranges = match_candidates[i]
            
            for j in range(len(ranges)):
                for first_column_index in range(ranges[j][0][0], ranges[j][0][1] + 1):
                    offset = ranges[j][1]
                    text_position = suffix_array[first_column_index] - offset
                    if text_position >= 0:
                        if ApproximateMatch(Pattern, Text[text_position: text_position + len(Pattern)], d):
                            PatternMatchIndices[Pattern].add(text_position)
    
    return PatternMatchIndices

def ApproximateMatch(Pattern, TextSubstring, d):
    mismatches = 0
    for i in range(len(Pattern)):
        if Pattern[i] != TextSubstring[i]:
            mismatches += 1
            if mismatches > d:
                return 0
    return 1
            

def SeededPattern(Pattern, d):
    k = int(len(Pattern) / (d + 1))
    seeds = []
    for i in range(d):
        seeds.append((Pattern[k * i: k * (i+1)], k * i))
    seeds.append((Pattern[k * d:], k * d))
    return seeds

def SuffixArray(Text):
    suffixes = []
    for i in range(len(Text)):
        suffixes.append((Text[i:], i))
    
    suffixes.sort()
    
    return [pair[1] for pair in suffixes]

def BWTfromSuffixArray(Text, suffix_array):
    Transform = ""
    for index in suffix_array:
        if index == 0:
            Transform += Text[-1]
        else:
            Transform += Text[index - 1]
    return Transform

def BetterBWMatchingPatterns(Transform, Patterns, d):
    Count = defaultdict(list)
    FirstOccurrence = dict()
    lastFreqs = defaultdict(int)
    symbols = list()
    
    for i in range(len(Transform)):
        symbol = Transform[i]
        if symbol not in symbols:
            symbols.append(symbol)
        
        if len(Count[symbol]) == 0:
            Count[symbol] = [0] * (i+1)
        elif len(Count[symbol]) < i + 1:
            Count[symbol] += [Count[symbol][-1]] * (i + 1 - len(Count[symbol]))
            
        Count[symbol] += [Count[symbol][-1] + 1]
        
    for symbol in Count:
        if len(Count[symbol]) < len(Transform) + 1:
            Count[symbol] += [Count[symbol][-1]] * (len(Transform) + 1 - len(Count[symbol]))
            
    symbols.sort()
    count_symbols = 0
    for symbol in symbols:
        if symbol == '$':
            FirstOccurrence.update({symbol: 0})
            count_symbols += Count[symbol][-1]
        else:
            FirstOccurrence.update({symbol: count_symbols})
            count_symbols += Count[symbol][-1]
            
    SubstringMatches = [0] * len(Patterns)
    for i in range(len(Patterns)):
        Pattern = Patterns[i]
        seed_matches = []
        pattern_index = -1
        for seed in SeededPattern(Pattern, d):
            seed_match = BetterBWMatching(FirstOccurrence, Transform, seed[0], Count)
            if seed_match:
                #pattern_index = seed[1]
                seed_matches.append((seed_match, seed[1]))
        
        SubstringMatches[i] = seed_matches
    
    return SubstringMatches

def BetterBWMatching(FirstOccurrence, LastColumn, Pattern, Count):
    top = 0
    bottom = len(LastColumn) - 1
    
    while top <= bottom:
        if len(Pattern):
            symbol = Pattern[-1]
            Pattern = Pattern[:-1]
            foundFlag = 0
            
            for i in range(top, bottom + 1):
                if symbol == LastColumn[i]:
                    top = FirstOccurrence[symbol] + Count[symbol][top]
                    bottom = FirstOccurrence[symbol] + Count[symbol][bottom + 1] - 1
                    foundFlag = 1
                    break
            
            if not foundFlag:
                return 0
            
        else:
            return top, bottom
    
    return 0
    
    
Matches = MultipleApproximatePatternMatching(Text, Patterns, d)
for pattern in Matches:
    out = pattern + ": "
    indices_list = list(Matches[pattern])
    indices_list.sort()
    for index in indices_list:
        out += str(index) + " "
    print(out.strip(" "))

[10000, 1947, 1881, 1948, 1882, 1949, 3787, 9103, 1883, 3993, 1950, 1341, 3788, 9104, 6915, 1884, 4222, 3994, 4205, 1876, 9835, 1951, 3711, 5304, 1342, 3789, 8617, 2153, 5782, 9105, 1916, 1800, 5666, 6916, 1885, 4223, 465, 3995, 4716, 4206, 5380, 3799, 1877, 9836, 4215, 1952, 3712, 5140, 2125, 5305, 823, 1343, 3790, 733, 7230, 7069, 8618, 8896, 615, 557, 769, 6260, 352, 2154, 3674, 3186, 8374, 7729, 6082, 5201, 5783, 740, 8486, 9106, 2758, 1917, 5912, 1801, 3855, 7175, 6632, 5667, 6917, 6492, 9649, 1886, 1385, 4974, 7005, 4224, 466, 1685, 4758, 3996, 1336, 3352, 4717, 4113, 5467, 4185, 9782, 4207, 6400, 1674, 5413, 6533, 8113, 8445, 3158, 250, 5381, 6886, 3243, 3800, 8014, 7745, 9898, 7187, 5020, 5057, 3284, 9584, 8921, 5209, 9173, 1805, 9933, 5068, 2865, 8236, 4805, 6450, 7219, 9978, 1878, 8614, 9779, 9837, 8268, 4216, 6348, 1738, 1953, 3329, 4838, 3713, 6157, 3419, 887, 5141, 8388, 7546, 3846, 4812, 4070, 5926, 4529, 7307, 2126, 3555, 5306, 7537, 9840, 824, 657, 8204, 1344, 3685, 175

GTGTGCGGGTTACCTAGGGCCACTGTTGAGATAGCCGCCAGCGCATGATGCTCGTGAGATCTATACGAGTTA: 8540
CAGCGCCGGTCAAAACGTGCTTAATATGCGTTTTAGAATCGACATGGCGAGCAAGGACAGGGTCCTGTTGTG: 5771
CCGTTTTGAAGCATAGCCAAAGAGTGTGGCTGAGCTGATCTTGACCTCATTTGGTCGTGATGGGACCGATTG: 1367
TCGTGGAGAGATCATGCGGTTGCCACACCAGCCTACCTTGATATCCGCCAATCTAAGGCGGGCCGAGGCCACAT: 3912
CCTCGAACGCTTCCCATTCCAACATACATTCATGCGCGCCGTTTTATTTGAACGCGGCATCTTTCAAGGCA: 4509
AAAAAAGAGGTCTCGATCACTCGATAGGCGTTCAAAACTTTCATATGCTCCGTCTGTATCGTACA: 1883
TTTGCAGGTCCCAGTTCGCGAGCATAACGCCCGCGATATTATTGTCTGTACCATGATACTCTAGTG: 7777
GAGTCCTTCATTTAGAACGCGATAAAGACGATAACGCGCCACTTACTTTCCTAGCGCAGCGG: 6469
CGCTAAGGATATGTTATGCAAGTAATCTAGGCCCACGCAAGTTCTCGTCACATGCCCAATGGAGCGAGCTCATGT: 5520
CTTGCGGAGTGTTTATGACTCCGCGAATATCGCGACTCTTCTCGAGACGGATTATGTGAACAGA: 8330
GGCATCAGTATTTAATACTACCGGTTAAGTTTAGCTAGACGGCGTAATACTTCTATGTGCTT: 6094
GCACCGCGAGCAAGCGGCACAATCTCGCTTGTAAGTGCGTTGGAGGAATTCGGCAGATGTTAGGTGTAGACC: 2820
CTTGCTACAGATCCTACTTTGGCGCATCCATCGAGAGGAGCCGACACCCCCATTTATGACCTGGGTTGGTAGCAC: 6761
AAACATA